In [1]:
pip install intrinio_sdk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import utils
import pandas as pd
import nltk
import numpy as np
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hag75415\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
from __future__ import print_function
import time
import intrinio_sdk
from intrinio_sdk.rest import ApiException
from pprint import pprint

intrinio_sdk.ApiClient().configuration.api_key['api_key'] = 'OmRkZTJkOWRkYzBlNjg4MjUwYzIxZjE2ODgwYjRjYmNi'

company_api = intrinio_sdk.CompanyApi()

identifier = 'AAPL' # str | A Company identifier (Ticker, CIK, LEI, Intrinio ID)
page_size = 100 # int | The number of results to return (optional) (default to 100)
next_page = '' # str | Gets the next page of data from a previous API call (optional)

api_response_list = []
condition = True
while condition:

    try:
        api_response = company_api.get_company_news(identifier, page_size=page_size, next_page=next_page)
        api_response_list.append(api_response)
        next_page = api_response.next_page
        pprint('got page')
#         time.sleep(63)
    except ApiException as e:
        print("Exception when calling CompanyApi->get_company_news: %s\r\n" % e)
        condition = False

'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'
'got page'

In [4]:
#Extract URL, Title, Date and Summary of each piece of News
url_list = []
title_list = []
date_list = []
summary_list = []

cnt = 0
last = ''

for page_100 in api_response_list:
    for article in page_100.news:
        
        if last == article.title:
            cnt+= 1
            pass 
        last = article.title
        
        
        url_list.append(article.url)
        title_list.append(article.title)
        date_list.append(article.publication_date)
        summary_list.append(article.summary)

In [5]:
#Assigning news to dates within stock trading hours
#If news was released after 4pm (when the stock market closes), the news affects next day's stock price
from datetime import timedelta
new_date = []
for date_item in date_list:
    if date_item.hour < 15:
        new_date.append(date_item)
    elif date_item.hour > 16:
        new_date.append(date_item - timedelta(days=1))
    else:
        new_date.append(np.nan)

In [6]:
#Dataframe with each New's URL, Title, Date and Summary
new_df = pd.DataFrame ({'url':url_list,
                       'title':title_list,
                       'date':new_date,
                       'summary':summary_list})

new_df.to_csv('News DataFrame.csv')

In [7]:
#Read Apple's Stock data retrieved from Bloomberg
stock_data = pd.read_csv("Final Stock Data.csv", encoding = "ISO-8859-1")

In [8]:
new_df = pd.read_csv("News DataFrame.csv", encoding = "ISO-8859-1")

In [9]:
#Columns of interest 
stock_data = stock_data[['Dates','Date Close Price (n)', 'Next Day Close Price (n+1)', 'Difference in Price (n+1) - (n)', 'Percentage of Difference to Close Price', 'Label']]

In [10]:
#Match News data to Stock data
new_df['date']=pd.to_datetime(new_df['date']).dt.strftime('%Y-%m-%d')

In [11]:
stock_data['date']=pd.to_datetime(stock_data['Dates']).dt.strftime('%Y-%m-%d')

In [12]:
#Merging News and Stock data to one table
result = pd.merge(new_df, stock_data, on='date')
result

result.to_csv('Result 1.csv')